# Part 2: Tools i Function Calling
## JCM Technologies - Departament de R+D

<table style="margin: 0; text-align: left; width:100%; border: 2px solid #2e86ab;">
    <tr>
        <td style="width: 150px; vertical-align: middle; padding: 20px;">
            <img src="https://raw.githubusercontent.com/googlefonts/noto-emoji/main/png/128/emoji_u1f6e0.png" width="120" style="display: block;" />
        </td>
        <td style="padding: 20px;">
            <h2 style="color:#2e86ab; margin: 0;">Objectius d'aprenentatge</h2>
            <ul style="color:#333; margin-top: 10px;">
                <li>Entendre què són les <strong>tools</strong> i per què són essencials</li>
                <li>Crear <strong>tools personalitzades</strong> per als teus agents</li>
                <li>Veure com els agents <strong>decideixen</strong> quina tool usar</li>
                <li>Interpretar <strong>traces</strong> per debugar agents</li>
                <li>Crear sistemes <strong>multi-agent</strong> col·laboratius</li>
            </ul>
            <p style="color:#ff7800; margin: 10px 0 0 0;"><strong>Durada estimada:</strong> 45-60 minuts</p>
        </td>
    </tr>
</table>

---
## 📚 Continguts

1. **Què són les Tools?** (10 min)
2. **Primera Tool:** Llegir fitxers Excel (15 min)
3. **Segona Tool:** Consultar API de preus (10 min)
4. **Traces en profunditat** (10 min)
5. **Múltiples agents col·laborant** (15 min)

---
## 1. Què són les Tools?

A la Part 1 vam veure agents que **només parlaven**. Ara els donarem **mans** per interactuar amb el món real.

### 1.1 El problema dels agents sense tools

Recordes l'agent de la Part 1 que analitzava un BOM?

```python
# Li passàvem el BOM com a text dins la task
task = Task(
    description=f"Analitza aquest BOM: {bom_simple}",
    ...
)
```

**Problemes d'aquest enfocament:**
1. ❌ El BOM ha d'estar **dins el codi** (no pot llegir fitxers)
2. ❌ Si el BOM és gran, ocupa tot el context del LLM
3. ❌ No pot consultar **dades externes** (preus, stock, etc.)
4. ❌ No pot **actualitzar** informació (guardar resultats, enviar emails, etc.)

**En resum:** L'agent només pot **pensar**, no pot **actuar**!

### 1.2 La solució: Function Calling (Tools)

**Function Calling** és la capacitat d'un LLM de:
1. Decidir **quan** necessita usar una funció
2. Escollir **quina** funció usar
3. Generar els **paràmetres** correctes
4. Interpretar el **resultat**
5. **Iterar** si cal més informació

**Esquema:**
<p align="center">
  <img src="diagrams/esquema_function_calling.png" width="350">
  <br>
  <em>Diagrama de blocs</em>
</p>

**Diferència clau:**
- **Abans:** L'agent **rep** tota la informació
- **Ara:** L'agent **busca** la informació que necessita

### 1.3 Com funciona per sota?

Quan dones tools a un agent, CrewAI fa això automàticament:

**1. Descriu les tools al LLM:**
```json
{
  "name": "read_excel",
  "description": "Llegeix un fitxer Excel i retorna les dades",
  "parameters": {
    "filepath": {"type": "string", "description": "Camí al fitxer"}
  }
}
```

**2. El LLM decideix si necessita una tool:**
```json
{
  "action": "use_tool",
  "tool": "read_excel",
  "parameters": {"filepath": "data/bom.xlsx"}
}
```

**3. CrewAI executa la tool Python:**
```python
result = read_excel(filepath="data/bom.xlsx")
```

**4. CrewAI retorna el resultat al LLM:**
```json
{
  "tool_result": "DataFrame amb 10 components..."
}
```

**5. El LLM decideix el següent pas:**
- Usar una altra tool?
- Processar la informació?
- Retornar la resposta final?

**Tot això passa automàticament!** Tu només defineixes les tools.

### 1.4 Tipus de tools a CrewAI

**Tools predefinides** (de la llibreria `crewai-tools`):
- `FileReadTool` - Llegir fitxers de text
- `DirectoryReadTool` - Llistar fitxers d'un directori
- `SerperDevTool` - Buscar a Google
- `WebsiteSearchTool` - Buscar en un website específic
- `ScrapeWebsiteTool` - Extreure contingut d'una pàgina web
- I moltes més...

**Tools personalitzades** (les que crearem nosaltres):
- Llegir Excel amb pandas
- Consultar una API interna
- Accedir a una base de dades
- Enviar emails
- Qualsevol cosa que puguis fer amb Python!

**Documentació:** https://docs.crewai.com/tools

---
## 2. Primera Tool: Llegir Fitxers Excel

Crearem una tool que permeti als agents llegir BOMs reals des d'Excel.

### 2.1 Preparar un BOM d'exemple en Excel

Primer, creem un fitxer Excel amb un BOM de prova:

In [ ]:
import pandas as pd
import os

# Crear directori per dades si no existeix
os.makedirs('data', exist_ok=True)

# BOM d'exemple
bom_data = {
    'Part Number': [
        'RES-10K-1/4W',
        'RES-1K-1/4W', 
        'CAP-100N-CER',
        'CAP-10U-ELEC',
        'LED-RED-5MM',
        'LED-GREEN-5MM',
        'RELAY-5V-10A',
        'IC-ATMEGA328P',
        'CONN-RJ45',
        'PCB-CTRL-V2.1'
    ],
    'Description': [
        'Resistència 10kΩ 1/4W 5%',
        'Resistència 1kΩ 1/4W 5%',
        'Condensador ceràmic 100nF 50V',
        'Condensador electrolític 10µF 25V',
        'LED vermell 5mm difús',
        'LED verd 5mm difús',
        'Relé 5V SPDT 10A',
        'Microcontrolador ATmega328P DIP-28',
        'Connector RJ45 8P8C',
        'PCB Control Board v2.1'
    ],
    'Quantity': [5, 3, 4, 2, 2, 1, 3, 1, 1, 1],
    'Unit': ['pcs', 'pcs', 'pcs', 'pcs', 'pcs', 'pcs', 'pcs', 'pcs', 'pcs', 'pcs'],
    'Manufacturer': [
        'Yageo',
        'Yageo',
        'Murata',
        'Nichicon',
        'Kingbright',
        'Kingbright',
        'Omron',
        'Microchip',
        'TE Connectivity',
        'JCM Tech'
    ]
}

df_bom = pd.DataFrame(bom_data)

# Guardar a Excel
df_bom.to_excel('data/bom_control_board_v2.1.xlsx', index=False)

print("✅ Fitxer Excel creat: data/bom_control_board_v2.1.xlsx")
print("\n📋 Contingut del BOM:")
display(df_bom)

### 2.2 Crear la Tool personalitzada

Ara crearem una tool que l'agent pugui usar per llegir aquest Excel.

**Hi ha 2 maneres de crear tools a CrewAI:**

**Opció A: Funció Python decorada** (més simple)
```python
from crewai.tools import tool

@tool
def my_tool(param: str) -> str:
    """Descripció de la tool"""
    return result
```

**Opció B: Classe BaseTool** (més control)
```python
from crewai.tools import BaseTool

class MyTool(BaseTool):
    name: str = "My Tool"
    description: str = "What it does"
    
    def _run(self, param: str) -> str:
        return result
```

**Usarem l'Opció A** (més simple i directa):

In [ ]:
from crewai.tools import tool
import pandas as pd

@tool("Read Excel BOM")
def read_excel_bom(filepath: str) -> str:
    """
    Llegeix un fitxer Excel que conté un Bill of Materials (BOM).
    
    Args:
        filepath: Camí al fitxer Excel (ex: 'data/bom.xlsx')
        
    Returns:
        String amb el contingut del BOM en format llegible
    """
    try:
        # Llegir Excel
        df = pd.read_excel(filepath)
        
        # Convertir a text llegible
        output = f"BOM llegit de {filepath}:\n\n"
        output += f"Total de components: {len(df)}\n\n"
        output += "Components:\n"
        output += "=" * 80 + "\n"
        
        for idx, row in df.iterrows():
            output += f"{idx+1}. {row['Part Number']}\n"
            output += f"   Descripció: {row['Description']}\n"
            output += f"   Quantitat: {row['Quantity']} {row['Unit']}\n"
            output += f"   Fabricant: {row['Manufacturer']}\n"
            output += "-" * 80 + "\n"
        
        return output
        
    except FileNotFoundError:
        return f"ERROR: No s'ha trobat el fitxer {filepath}"
    except Exception as e:
        return f"ERROR llegint el fitxer: {str(e)}"

print("✅ Tool 'read_excel_bom' creada!")

**💡 Parts importants de la tool:**

1. **Decorator `@tool`:** Converteix la funció en una tool de CrewAI
2. **Nom descriptiu:** "Read Excel BOM" (apareix a les traces)
3. **Docstring:** Descriu què fa i els paràmetres (el LLM llegeix això!)
4. **Type hints:** `filepath: str -> str` (ajuda al LLM a saber els tipus)
5. **Gestió d'errors:** Retorna missatges clars si alguna cosa falla
6. **Output llegible:** Format de text que el LLM pot processar

### 2.3 Provar la tool manualment

In [ ]:
# Provar la tool directament
result = read_excel_bom._run('data/bom_control_board_v2.1.xlsx')
print(result)

### 2.4 Agent amb la Tool

Ara crearem un agent que **pot usar** aquesta tool:

In [ ]:
from crewai import Agent, Task, Crew
from dotenv import load_dotenv

load_dotenv(override=True)

# Agent amb accés a la tool
agent_bom_reader = Agent(
    role="Analitzador de BOMs amb accés a fitxers",
    goal="Llegir i analitzar BOMs des de fitxers Excel",
    backstory="""
    Ets un especialista en gestió de BOMs a JCM Technologies.
    
    Tens accés a eines per llegir fitxers Excel amb BOMs.
    Pots analitzar components, identificar categories, i detectar
    possibles problemes o inconsistències.
    
    Sempre uses les eines disponibles per accedir a la informació
    abans de fer qualsevol anàlisi.
    """,
    tools=[read_excel_bom],  # ← Aquí assignem la tool!
    llm="gemini/gemini-2.5-flash",
    verbose=True
)

print("✅ Agent amb tool creat!")

**💡 La clau: `tools=[read_excel_bom]`**

Això li diu a l'agent:
- "Tens aquesta eina disponible"
- "Pots decidir quan usar-la"
- "Genera els paràmetres correctes per cridar-la"

### 2.5 Task que requereix usar la tool

In [ ]:
# Task que NECESSITA la tool
task_analyze_excel_bom = Task(
    description="""
    Analitza el BOM que està al fitxer Excel 'data/bom_control_board_v2.1.xlsx'.
    
    Has de:
    1. Llegir el fitxer Excel per obtenir els components
    2. Identificar quants components diferents hi ha
    3. Agrupar-los per fabricant
    4. Identificar possibles components crítics (microcontroladors, relés)
    5. Fer un resum executiu
    """,
    expected_output="""
    Un resum executiu que inclogui:
    - Total de components
    - Distribució per fabricant
    - Components crítics identificats
    - Observacions rellevants
    """,
    agent=agent_bom_reader
)

print("✅ Task creada!")

### 2.6 Executar i observar!

Ara ve la part interessant. L'agent **decidirà automàticament** usar la tool:

In [ ]:
# Crear i executar el crew
crew_with_tools = Crew(
    agents=[agent_bom_reader],
    tasks=[task_analyze_excel_bom],
    verbose=True
)

print("🚀 Executant crew amb tools...\n")
print("=" * 70)
print("\n⚠️ OBSERVA LES TRACES: Veuràs com l'agent decideix usar la tool!\n")
print("=" * 70)

result = crew_with_tools.kickoff()

print("\n" + "=" * 70)
print("✅ Executat amb èxit!")
print("=" * 70)
print("\n📄 RESULTAT FINAL:\n")
print(result)

### 💡 Què hauries d'haver vist a les traces:

```
[Agent] Thinking...
[Agent] Action: Read Excel BOM
[Agent] Action Input: {"filepath": "data/bom_control_board_v2.1.xlsx"}
[Tool] Executing Read Excel BOM...
[Tool] Result: BOM llegit de data/bom_control_board_v2.1.xlsx...
[Agent] Processing tool result...
[Agent] Thinking about next step...
[Agent] Final Answer: ...
```

**L'agent ha:**
1. ✅ Entès que necessitava llegir el fitxer
2. ✅ Decidit usar la tool `read_excel_bom`
3. ✅ Generat el paràmetre correcte (`filepath`)
4. ✅ Processat el resultat de la tool
5. ✅ Generat l'anàlisi final

**Tot això sense que li hagis dit explícitament com fer-ho!** 🤯

---
## 3. Segona Tool: Consultar API de Preus (Mockup)

Ara crearem una segona tool que simula consultar una API de preus de components.

### 3.1 Per què Mock?

Per aquesta formació, crearem una API **simulada** (mockup) perquè:
- ✅ No depenguem de connexió externa
- ✅ Podem controlar els resultats
- ✅ És més ràpid
- ✅ Mateix concepte que una API real

**Més endavant** (Part 3), podríeu integrar APIs reals de proveïdors!

### 3.2 Crear la Tool de Preus Mockup

In [ ]:
import random
import time

@tool("Get Component Price")
def get_component_price(part_number: str) -> str:
    """
    Consulta el preu d'un component electrònic.
    Aquesta és una versió mock (simulada) per a la formació.
    
    Args:
        part_number: Codi del component (ex: 'RES-10K-1/4W')
        
    Returns:
        String amb informació de preu i disponibilitat
    """
    # Simular latència d'API
    time.sleep(0.5)
    
    # Base de dades mock de preus
    mock_prices = {
        'RES-10K-1/4W': {'price': 0.05, 'stock': 5000, 'supplier': 'DigiKey'},
        'RES-1K-1/4W': {'price': 0.04, 'stock': 3000, 'supplier': 'DigiKey'},
        'CAP-100N-CER': {'price': 0.08, 'stock': 2000, 'supplier': 'Mouser'},
        'CAP-10U-ELEC': {'price': 0.15, 'stock': 1500, 'supplier': 'Mouser'},
        'LED-RED-5MM': {'price': 0.12, 'stock': 8000, 'supplier': 'Farnell'},
        'LED-GREEN-5MM': {'price': 0.13, 'stock': 7500, 'supplier': 'Farnell'},
        'RELAY-5V-10A': {'price': 2.50, 'stock': 500, 'supplier': 'RS Components'},
        'IC-ATMEGA328P': {'price': 3.20, 'stock': 200, 'supplier': 'DigiKey'},
        'CONN-RJ45': {'price': 0.85, 'stock': 1200, 'supplier': 'Mouser'},
        'PCB-CTRL-V2.1': {'price': 15.00, 'stock': 100, 'supplier': 'JCM Tech'}
    }
    
    # Buscar el component
    if part_number in mock_prices:
        info = mock_prices[part_number]
        return f"""
Informació de preu per {part_number}:
- Preu unitari: €{info['price']:.2f}
- Stock disponible: {info['stock']} unitats
- Proveïdor: {info['supplier']}
- Data: 2025-01-02
"""
    else:
        # Generar preu aleatori si no està a la BD
        random_price = round(random.uniform(0.10, 5.00), 2)
        return f"""
Informació de preu per {part_number}:
- Preu unitari: €{random_price:.2f}
- Stock disponible: Consultar amb proveïdor
- Proveïdor: Genèric
- NOTA: Component no trobat a la BD principal, preu estimat
"""

print("✅ Tool 'get_component_price' creada!")

### 3.3 Provar la tool manualment

In [ ]:
# Provar amb un component conegut
result = get_component_price._run('RELAY-5V-10A')
print(result)

# Provar amb un component no conegut
result2 = get_component_price._run('COMPONENT-DESCONEGUT')
print(result2)

### 3.4 Agent amb múltiples tools

Ara crearem un agent que pot usar **AMBDUES** tools!

In [ ]:
# Agent amb múltiples tools
agent_bom_pricer = Agent(
    role="Analitzador de BOMs i Preus",
    goal="Llegir BOMs i obtenir informació de preus dels components",
    backstory="""
    Ets un especialista en càlcul de costos a JCM Technologies.
    
    Tens accés a:
    - Eina per llegir fitxers Excel amb BOMs
    - Eina per consultar preus de components
    
    La teva feina és analitzar BOMs i calcular costos estimats,
    identificant components cars i possibles alternatives.
    
    Sempre uses les eines disponibles de manera eficient:
    primer llegir el BOM, després consultar preus dels components
    més rellevants o cars.
    """,
    tools=[read_excel_bom, get_component_price],  # ← 2 tools!
    llm="gemini/gemini-2.5-flash",
    verbose=True,
    max_iter=10,  # ← Màxim d'iteracions
    max_rpm=10    # ← Requests per minut
)

print("✅ Agent amb múltiples tools creat!")

### 3.5 Task complexa que requereix ambdues tools

In [ ]:
task_bom_cost_analysis = Task(
    description="""
    Fes una anàlisi de costos del BOM 'data/bom_control_board_v2.1.xlsx'.
    
    Has de:
    1. Llegir el BOM del fitxer Excel
    2. Identificar els 3 components més cars (per intuïció tècnica)
    3. Consultar el preu de cada un d'aquests 3 components
    4. Calcular el cost estimat NOMÉS d'aquests 3 components
    5. Fer una recomanació sobre possibles optimitzacions de cost
    
    IMPORTANT: No cal consultar TOTS els preus, només els 3 més rellevants.
    """,
    expected_output="""
    Un informe de cost que inclogui:
    - Número total de components al BOM
    - Els 3 components més cars amb preus detallats
    - Cost estimat d'aquests 3 components (preu x quantitat)
    - Recomanacions per optimitzar costos
    """,
    agent=agent_bom_pricer
)

print("✅ Task complexa creada!")

### 3.6 Executar i observar les decisions!

Això és fascinant. L'agent decidirà:
1. Quina tool usar primer
2. Quants cops usar cada tool
3. Amb quins paràmetres
4. Quan ha acabat

In [ ]:
crew_multi_tools = Crew(
    agents=[agent_bom_pricer],
    tasks=[task_bom_cost_analysis],
    verbose=True
)

print("🚀 Executant agent amb múltiples tools...\n")
print("=" * 70)
print("\n⚠️ OBSERVA: L'agent decidirà quina tool usar i quan!\n")
print("=" * 70)

result = crew_multi_tools.kickoff()

print("\n" + "=" * 70)
print("✅ Anàlisi de cost completada!")
print("=" * 70)
print("\n📊 RESULTAT:\n")
print(result)

### 💡 Reflexió sobre el que acabes de veure

**L'agent ha pres decisions autònomes:**
1. ✅ Va usar primer `read_excel_bom` per obtenir el BOM
2. ✅ Va identificar (amb el seu coneixement) quins components són cars
3. ✅ Va cridar `get_component_price` 3 vegades (una per cada component)
4. ✅ Va processar els resultats i va fer càlculs
5. ✅ Va generar recomanacions

**Tot això sense que li diguessis:**
- ❌ "Primer usa aquesta tool"
- ❌ "Després usa aquesta altra"
- ❌ "Crida-la 3 vegades"

**L'agent ho va decidir tot sol!** Això és el poder del function calling. 🚀

**Pregunta per pensar:**
- Què passaria si ara li dones una tool per "buscar components alternatius"?
- L'agent sabria quan usar-la?
- Podria decidir buscar alternatives per components cars?

*Resposta:* Sí! L'agent adaptaria la seva estratègia basant-se en les tools disponibles!

## 4. Traces en Profunditat: Aprendre a Llegir i Debugar

Ara que has vist els agents en acció, anem a dominar l'art de **llegir traces** per entendre i debugar agents.

### 4.1 Anatomia d'una Trace

Quan executes un crew amb `verbose=True`, CrewAI mostra traces detallades. Anem a descompondre cada part:

#### 📦 Capçalera: Crew Execution Started

```
╭──────────────────────── Crew Execution Started ────────────────────────╮
│  Name: crew                                                            │
│  ID: 8f03b202-1c46-4ba6-b488-12288afb9ecf                              │
╰────────────────────────────────────────────────────────────────────────╯
```

**Què significa:**
- Nom del crew (pots personalitzar-lo)
- ID únic de l'execució (útil per tracking)

**Per a què serveix:**
- Identificar execucions diferents
- Logs i debugging

#### 🤖 Agent Started

```
╭─────────────────────── 🤖 Agent Started ───────────────────────╮
│  Agent: Analitzador de BOMs i Preus                            │
│  Task: Fes una anàlisi de costos del BOM...                    │
╰────────────────────────────────────────────────────────────────╯
```

**Què significa:**
- Quin agent ha començat a treballar
- Quina task li han assignat

**Per a què serveix:**
- Confirmar que l'agent correcte té la task correcta
- En sistemes multi-agent, veure qui fa què

#### 🔧 Agent Tool Execution

```
╭──────────────────────── 🔧 Agent Tool Execution ────────────────────────╮
│  Agent: Analitzador de BOMs i Preus                                     │
│  Thought: Action: Read Excel BOM                                        │
│  Using Tool: Read Excel BOM                                             │
╰─────────────────────────────────────────────────────────────────────────╯
```

**Què significa:**
- **Thought:** El raonament de l'agent (per què fa això)
- **Using Tool:** Quina tool ha decidit usar

**Per a què serveix:**
- **CRÍTIC PER DEBUGGING:** Veure si l'agent pren bones decisions
- Entendre el raonament darrere cada acció

**⚠️ Important:** Els **paràmetres i resultats** de la tool estan **col·lapsats** per defecte per no saturar la sortida.

#### 🧠 Thinking...

```
🚀 Crew: crew
└── 📋 Task: 76f532ad...
    Status: Executing Task...
    └── 🧠 Thinking...
```

**Què significa:**
- L'agent està processant informació
- Pot estar decidint quina tool usar
- Pot estar preparant la resposta final

**Per a què serveix:**
- Indicador de progrés
- Si es queda aquí molt temps → potser hi ha un problema

#### 🔢 Comptador de Tools

```
🚀 Crew: crew
└── 📋 Task: 76f532ad...
    Status: Executing Task...
    ├── 🔧 Used Read Excel BOM (2)
    ├── 🔧 Used Get Component Price (1)
    ├── 🔧 Used Get Component Price (2)
    └── 🔧 Used Get Component Price (3)
```

**Què significa:**
- Quantes vegades s'ha usat cada tool
- El número entre parèntesis és el **comptador acumulat**

**Per a què serveix:**
- **MOLT ÚTIL:** Detectar si l'agent està cridant massa vegades una tool
- Verificar que ha cridat totes les tools necessàries
- Detectar loops infinits

**Exemple de problema:**
```
├── 🔧 Used Get Component Price (50)
```
⚠️ 50 crides? Potser hi ha un loop infinit!

#### ✅ Agent Final Answer

```
╭───────────────────── ✅ Agent Final Answer ─────────────────────╮
│  Agent: Analitzador de BOMs i Preus                            │
│  Final Answer:                                                 │
│  Informe d'anàlisi de costos...                                │
╰─────────────────────────────────────────────────────────────────╯
```

**Què significa:**
- L'agent ha acabat i té una resposta final
- Aquesta és la sortida que retornarà

**Per a què serveix:**
- Veure el resultat final abans que es processi
- **CLAU PER DEBUGGING:** Si està buit → problema!

#### 📊 Task Completion

```
╭──────────────────────── Task Completion ────────────────────────╮
│  Task Completed                                                │
│  Name: 76f532ad-ca31-4b73-b1ca-9da7e7550c04                    │
│  Agent: Analitzador de BOMs i Preus                           │
╰────────────────────────────────────────────────────────────────╯
```

**Què significa:**
- La task s'ha completat correctament
- Resum de qui la va fer

**Per a què serveix:**
- Confirmació d'èxit
- En sistemes multi-task, veure quines s'han completat

### 4.2 Patrons comuns a les traces

Aprendre a reconèixer patrons t'ajudarà a debugar més ràpid:

#### ✅ Patró CORRECTE: Workflow lògic

```
1. 🔧 Using Tool: Read Excel BOM
2. 🧠 Thinking... (processant dades del BOM)
3. 🔧 Using Tool: Get Component Price (component 1)
4. 🧠 Thinking... (processant preu)
5. 🔧 Using Tool: Get Component Price (component 2)
6. 🧠 Thinking... (processant preu)
7. 🔧 Using Tool: Get Component Price (component 3)
8. 🧠 Thinking... (calculant totals)
9. ✅ Final Answer: Informe complet
```

**Característiques:**
- ✅ Seqüència lògica
- ✅ Thinking entre tools (processant resultats)
- ✅ Final Answer present i complet

#### ❌ Patró PROBLEMÀTIC: Final Answer buit

```
1. 🔧 Using Tool: Read Excel BOM
2. 🔧 Using Tool: Get Component Price (1)
3. 🔧 Using Tool: Get Component Price (2)
4. ✅ Final Answer:
   (BUIT!)
```

**Diagnòstic:**
- ❌ L'agent va usar les tools
- ❌ Però no va generar resposta final

**Possibles causes:**
1. **Límit de tokens:** El model es va quedar sense espai per la resposta
2. **Error de validació:** Si tenies `output_pydantic`, potser no va poder generar l'esquema
3. **Instruccions confuses:** L'expected_output no estava clar

**Solucions:**
- Simplificar la task
- Fer l'expected_output més explícit
- Augmentar max_tokens (si és possible)

#### ❌ Patró PROBLEMÀTIC: Loop infinit

```
1. 🔧 Using Tool: Get Component Price (1)
2. 🔧 Using Tool: Get Component Price (2)
3. 🔧 Using Tool: Get Component Price (3)
...
25. 🔧 Using Tool: Get Component Price (25)
26. 🔧 Using Tool: Get Component Price (26)
ERROR: Max iterations reached
```

**Diagnòstic:**
- ❌ L'agent crida la mateixa tool una i altra vegada
- ❌ No arriba mai a Final Answer

**Possibles causes:**
1. **Tool retorna error:** L'agent reintenta constantment
2. **Expected output impossible:** L'agent busca info que no pot trobar
3. **Instruccions ambigües:** L'agent no sap quan parar

**Solucions:**
- Verificar que les tools funcionen correctament
- Afegir condició de parada clara a la task: "Consulta màxim 3 preus"
- Reduir `max_iter` per evitar loops llargs

#### ❌ Patró PROBLEMÀTIC: Tool mai usada

```
1. 🧠 Thinking...
2. 🧠 Thinking...
3. ✅ Final Answer: No he pogut accedir al fitxer...
```

**Diagnòstic:**
- ❌ L'agent NO va usar cap tool
- ❌ Va intentar respondre sense les eines necessàries

**Possibles causes:**
1. **Backstory poc clar:** L'agent no sap que té tools disponibles
2. **Task ambigua:** No està clar que necessita usar tools
3. **Nom/descripció de tool poc clara:** L'agent no entén què fa la tool

**Solucions:**
- Afegir al backstory: "Tens accés a eines per..."
- A la task: "Has de usar la tool X per..."
- Millorar la docstring de la tool

### 4.3 Habilitar tracing complet (opcional)

Si necessites **encara més detall**, CrewAI ofereix un sistema de tracing complet:

#### Opció 1: Habilitar al codi

```python
crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True,
    tracing=True  # ← Activa tracing complet
)
```

#### Opció 2: Habilitar globalment

Al fitxer `.env`:
```bash
CREWAI_TRACING_ENABLED=true
```

O al terminal:
```bash
crewai traces enable
```

#### Què ofereix el tracing complet?

- ✅ **Paràmetres complets** de cada tool call
- ✅ **Resultats complets** que retornen les tools
- ✅ **Prompts exactes** enviats al LLM
- ✅ **Respostes completes** del LLM
- ✅ **Timestamps** de cada operació
- ✅ **Visualització web** a https://app.crewai.com/crewai_plus/trace_batches

**⚠️ Inconvenient:** Genera MOLTA informació (pots saturar-te)

### 4.4 Consells pràctics de debugging

Basats en experiència real amb agents:

#### 🎯 Consell 1: Comença simple

Quan desenvolupis un agent nou:

```python
# ❌ NO començar amb això:
task = Task(
    description="Llegeix el BOM, consulta preus a 3 APIs diferents, 
                 compara amb històric, genera informe PDF..."
)

# ✅ Començar amb això:
task = Task(
    description="Llegeix el BOM i mostra el primer component"
)
```

**Per què:**
- Pots verificar que la tool funciona
- Veus com l'agent decideix usar-la
- Després afegeixes complexitat gradualment

#### 🎯 Consell 2: Sempre prova les tools manualment primer

```python
# Abans de donar-la a l'agent:
result = my_tool._run(param="test")
print(result)

# Verifica:
# - Funciona?
# - Retorna el format esperat?
# - Els errors es gestionen bé?
```

#### 🎯 Consell 3: Revisa els comptadors de tools

```
├── 🔧 Used Read Excel BOM (2)  ← ⚠️ Per què 2 vegades?
├── 🔧 Used Get Component Price (1)
```

Si veus números inesperats:
- Revisa el "Thought" per entendre per què
- Potser l'agent va rebre un error i va reintentar
- Potser les instruccions no eren clares

#### 🎯 Consell 4: Expected Output molt explícit

```python
# ❌ Vague:
expected_output="Un resum del BOM"

# ✅ Explícit:
expected_output="""
Format requerit:
## Resum BOM
- Total components: [número]
- Components cars: [llista amb preus]
- Cost estimat: €[total]
"""
```

Quan més explícit, millor resultat!

#### 🎯 Consell 5: Usa print statements a les tools

```python
@tool
def my_tool(param: str) -> str:
    """Tool description"""
    print(f"[DEBUG] Tool called with: {param}")  # ← Afegeix això!
    
    result = do_something(param)
    
    print(f"[DEBUG] Tool returning: {result[:100]}...")  # ← I això!
    
    return result
```

Això et permet veure exactament què passa dins les tools.

### 💡 Exercici pràctic de debugging

Torna a executar el crew de la secció 3.6 i fixa't en:

1. **Seqüència de tools:** En quin ordre les va usar?
2. **Comptadors:** Quantes vegades va usar cada tool?
3. **Thoughts:** Quin era el raonament en cada pas?
4. **Temps:** Quant va trigar cada tool? (mira els timestamps)

Això t'entrenarà per llegir traces ràpidament quan debugis els teus propis agents!

---
## 5. Múltiples Agents Col·laborant

Fins ara hem treballat amb **1 agent fent totes les tasques**. Ara veurem el **poder real** de CrewAI: **equips d'agents** col·laborant!

### 5.1 Per què múltiples agents?

**Analogia del món real:**

A JCM Technologies, quan analitzem un projecte nou:
- ❌ NO fem que UN enginyer faci tot (disseny + compres + producció + testing)
- ✅ Cada enginyer té la seva **especialitat** i **col·laboren**

**El mateix amb agents:**

```
Agent Generalista (fa tot):
- Llegeix BOM ✓
- Busca preus ✓ (però no és expert)
- Compara preus ✓ (però no és expert)
- Genera informe ✓ (però genèric)

vs

Equip d'Agents Especialitzats:
Agent 1: Expert en BOMs → Analitza components perfectament
Agent 2: Expert en preus → Troba millors ofertes
Agent 3: Expert en informes → Documents professionals
```

**Avantatges:**
- ✅ Cada agent és **expert** en la seva tasca
- ✅ **Millor qualitat** per tasca
- ✅ Més **escalable** (afegir nous experts)
- ✅ Més **mantenible** (canviar un agent sense afectar els altres)

### 5.2 Com CrewAI implementa els patrons agèntics

Recordes els **patrons agèntics** de la Part 1? Eren conceptes genèrics aplicables a qualsevol framework.

Ara veurem com **CrewAI implementa** alguns d'aquests patrons com a "Process" predefinits:

#### 📊 Comparativa: Patrons Part 1 vs Process CrewAI

**🔗 Prompt Chaining → `Process.sequential`**
- **Què fa:** Tasks executen una darrere l'altra. L'output d'una alimenta la següent.
- **Quan usar:** Workflow lineal amb dependències clares
- **Exemple:** Llegir BOM → Buscar preus → Generar informe

**🎭 Orchestrator-Worker → `Process.hierarchical`**  
- **Què fa:** Un agent "Manager" coordina i assigna tasques dinàmicament.
- **Quan usar:** Tasques complexes que requereixen coordinació adaptativa
- **Exemple:** Manager decideix què analitzar i qui ho fa

**⚡ Patró nou → `Process.consensus`**
- **Què fa:** Múltiples agents fan la mateixa tasca i es tria la millor resposta.
- **Quan usar:** Decisions crítiques que necessiten alta confiança
- **Exemple:** 3 agents avaluen risc d'un component, es tria la més conservadora

---

**💡 Diferència clau:**
- **Part 1:** Patrons **conceptuals** (com pensar sobre workflows)
- **Part 2:** **Implementacions** de CrewAI (com executar-los amb codi)

#### 🔗 Process.sequential en detall

**Implementa el patró Prompt Chaining de la Part 1:**

<p align="center">
  <img src="diagrams/sequential_diagram.png" width="350">
  <br>
  <em>Procés seqüencial</em>
</p>

**Com funciona:**
```python
task2 = Task(
    description="...",
    agent=agent2,
    context=[task1]  # ← Rep automàticament l'output de task1
)

crew = Crew(
    agents=[agent1, agent2],
    tasks=[task1, task2],
    process=Process.sequential  # ← Executa en ordre
)
```

**Exemple pràctic (el que farem aviat):**
1. Agent 1 llegeix BOM → Llista de components
2. Agent 2 busca preus (usa la llista) → Preus trobats
3. Agent 3 genera informe (usa components + preus) → Informe final

#### 👔 Process.hierarchical en detall

**Implementa el patró Orchestrator-Worker de la Part 1:**

<p align="center">
  <img src="diagrams/hierarchical_diagram.png" width="350">
  <br>
  <em>Procés hierarchical</em>
</p>

**Com funciona:**
```python
crew = Crew(
    agents=[agent1, agent2, agent3],  # Workers
    tasks=[task],
    process=Process.hierarchical,  # ← Manager coordina
    manager_llm="gemini/gemini-2.5-flash"  # LLM del manager
)
```

**Quan usar:**
- Tasques on no saps exactament quins passos cal fer
- El manager decideix dinàmicament qui fa què
- Més flexible però més difícil de debugar

**Nota:** No el veurem en aquesta formació (més avançat)

#### ⚡ Process.consensus en detall

**Patró nou (no cobert a la Part 1):**

<p align="center">
  <img src="diagrams/consensus_diagram.png" width="350">
  <br>
  <em>Procés hierarchical</em>
</p>

**Com funciona:**
- Tots els agents reben la mateixa task
- Cada agent dona la seva resposta
- CrewAI avalua i tria la millor

**Quan usar:**
- Decisions crítiques de disseny
- Validació de components crítics
- Quan necessites alta confiança

**Exemple:**
- 3 agents analitzen un component de seguretat
- Cada un avalua el risc
- Es tria l'avaluació més conservadora

**Nota:** Tampoc el veurem en aquesta formació (més avançat)

#### 🎯 Per a aquesta formació: Process.sequential

Usarem **`Process.sequential`** perquè:
- ✅ És el més comú en casos reals (80% dels usos)
- ✅ Fàcil d'entendre i debugar
- ✅ Predictible (sempre sabem l'ordre)
- ✅ Perfecte per workflows amb dependències clares

**El nostre cas d'ús (anàlisi de BOM) és perfecte per sequential:**
1. Primer cal llegir el BOM
2. Després buscar preus dels components identificats
3. Finalment generar l'informe amb tota la info

Cada pas depèn de l'anterior → Sequential! 🎯

### 5.3 Crear un equip de 2 agents: Analitzador + Pricer

Separarem les responsabilitats:

In [ ]:
from crewai import Agent, Task, Crew, Process
from dotenv import load_dotenv

load_dotenv(override=True)

# Agent 1: Especialista en analitzar BOMs
agent_bom_analyst = Agent(
    role="Expert Analitzador de BOMs",
    goal="Analitzar BOMs i identificar components crítics i cars",
    backstory="""
    Ets un enginyer electrònic senior amb 15 anys d'experiència a JCM Technologies.
    
    La teva especialitat és analitzar Bills of Materials i identificar:
    - Components crítics per al funcionament
    - Components cars que impacten el cost
    - Possibles problemes o inconsistències
    
    Tens accés a eines per llegir fitxers Excel amb BOMs.
    
    El teu anàlisi és sempre tècnic, precís i orientat a la qualitat.
    """,
    tools=[read_excel_bom],
    llm="gemini/gemini-2.5-flash",
    verbose=True
)

# Agent 2: Especialista en preus
agent_price_expert = Agent(
    role="Expert en Preus de Components",
    goal="Trobar i comparar preus de components electrònics",
    backstory="""
    Ets un especialista en compres amb 15 anys d'experiència.
    
    La teva expertesa és:
    - Trobar els millors preus per components
    - Identificar oportunitats d'estalvi
    - Conèixer proveïdors alternatius
    
    Tens accés a eines per consultar preus de components.
    
    Sempre proporciones informació detallada de preus i stock,
    i sugereixes alternatives quan els components són cars.
    """,
    tools=[get_component_price],
    llm="gemini/gemini-2.5-flash",
    verbose=True
)

print("✅ 2 agents especialitzats creats!")

**💡 Observa les diferències:**

**Agent 1 (BOM Analyst):**
- Role: "Expert Analitzador de BOMs"
- Tool: `read_excel_bom`
- Focus: Anàlisi tècnica

**Agent 2 (Price Expert):**
- Role: "Expert en Preus"
- Tool: `get_component_price`
- Focus: Compres i cost

**Cada agent és expert en el seu domini!**

### 5.4 Crear tasks per a cada agent

In [ ]:
# Task 1: Analitzar el BOM
task_analyze = Task(
    description="""
    Analitza el BOM del fitxer 'data/bom_control_board_v2.1.xlsx'.
    
    Has de:
    1. Llegir el fitxer Excel
    2. Identificar EXACTAMENT els 3 components més cars per intuïció tècnica
       (normalment: microcontroladors, relés, PCBs personalitzades)
    3. Per cada component identificat, proporciona:
       - Part Number EXACTE
       - Descripció
       - Quantitat necessària
       - Per què creus que és car
    
    IMPORTANT: La teva sortida serà usada pel següent agent per buscar preus.
    Proporciona NOMÉS els 3 part numbers més rellevants.
    """,
    expected_output="""
    Una llista dels 3 components més cars amb aquest format:
    
    ## Components Cars Identificats:
    
    1. **Part Number:** [codi exacte]
       - Descripció: [descripció]
       - Quantitat: [número]
       - Raó: [per què és car]
    
    2. **Part Number:** [codi exacte]
       ...
    
    3. **Part Number:** [codi exacte]
       ...
    """,
    agent=agent_bom_analyst
)

# Task 2: Buscar preus (DEPÈN de Task 1)
task_get_prices = Task(
    description="""
    Basant-te en l'anàlisi del BOM anterior, consulta els preus dels 
    3 components identificats.
    
    Per cada component:
    1. Usa la tool per obtenir el preu
    2. Calcula el cost total: preu_unitari × quantitat
    3. Avalua si el preu és raonable
    4. Suggereix possibles alternatives si el cost és alt
    
    IMPORTANT: Has de consultar els 3 components, un per un.
    """,
    expected_output="""
    Un informe de preus amb aquest format:
    
    ## Anàlisi de Preus
    
    ### Component 1: [Part Number]
    - Preu unitari: €X.XX
    - Quantitat: Y
    - Cost total: €Z.ZZ
    - Avaluació: [raonable/car/molt car]
    - Recomanació: [si cal alternatives]
    
    ### Component 2: ...
    
    ### Component 3: ...
    
    ## Cost Total Estimat: €XXX.XX
    
    ## Recomanacions d'Optimització:
    - [Recomanació 1]
    - [Recomanació 2]
    """,
    agent=agent_price_expert,
    context=[task_analyze]  # ← CLAU: Aquesta task depèn de l'anterior!
)

print("✅ 2 tasks seqüencials creades!")
print("\n💡 Nota: task_get_prices usa 'context=[task_analyze]'")
print("   Això significa que rebrà l'output de task_analyze com a input!")

**🔑 La clau: `context=[task_analyze]`**

Això li diu a CrewAI:
- Task 2 necessita l'output de Task 1
- Executa Task 1 primer
- Passa el resultat a Task 2 automàticament

### 5.5 Crear i executar el Crew multi-agent

In [ ]:
# Crew amb 2 agents treballant seqüencialment
crew_multi_agent = Crew(
    agents=[agent_bom_analyst, agent_price_expert],
    tasks=[task_analyze, task_get_prices],
    process=Process.sequential,  # ← Procés seqüencial
    verbose=True
)

print("✅ Crew multi-agent creat!")
print("\n📊 Configuració:")
print(f"  - Agents: {len(crew_multi_agent.agents)}")
print(f"  - Tasks: {len(crew_multi_agent.tasks)}")
print(f"  - Process: {crew_multi_agent.process}")

In [ ]:
# Executar el crew!
print("🚀 Executant crew multi-agent...\n")
print("=" * 70)
print("\n⚠️ OBSERVA: Veuràs 2 agents treballant en seqüència!\n")
print("=" * 70)

result = crew_multi_agent.kickoff()

print("\n" + "=" * 70)
print("✅ Crew multi-agent completat!")
print("=" * 70)
print("\n📊 RESULTAT FINAL:\n")
print(result)

### 💡 Què hauries d'observar a les traces:

```
╭─────────────────── 🤖 Agent Started ────────────────────╮
│  Agent: Expert Analitzador de BOMs                     │
│  Task: Analitza el BOM...                              │
╰─────────────────────────────────────────────────────────╯

[Agent 1 usa read_excel_bom]
[Agent 1 genera la llista de 3 components]

╭─────────────────── Task Completion ──────────────────────╮
│  Task 1 Completed                                        │
╰──────────────────────────────────────────────────────────╯

╭─────────────────── 🤖 Agent Started ────────────────────╮
│  Agent: Expert en Preus de Components                  │
│  Task: Basant-te en l'anàlisi anterior...              │
╰─────────────────────────────────────────────────────────╯

[Agent 2 rep l'output d'Agent 1]
[Agent 2 usa get_component_price 3 vegades]
[Agent 2 genera l'informe final]

╭─────────────────── Task Completion ──────────────────────╮
│  Task 2 Completed                                        │
╰──────────────────────────────────────────────────────────╯
```

**Veus la col·laboració?**
- ✅ Agent 1 fa la seva feina (analitzar)
- ✅ Agent 2 rep l'output d'Agent 1
- ✅ Agent 2 fa la seva feina (preus)
- ✅ El resultat final combina ambdues experteses!

### 5.6 Avantatges vs un sol agent

**Compara aquests dos enfocaments:**

#### Enfocament 1: Agent únic (el que vam fer abans)

```python
agent_todo = Agent(
    role="Analitzador i Pricer",  # Fa de tot!
    tools=[read_excel_bom, get_component_price]
)
```

**Problemes:**
- ❌ Backstory genèric ("ets expert en tot")
- ❌ Pot oblidar usar alguna tool
- ❌ Menys enfocat en cada tasca
- ❌ Difícil de mantenir si creix

#### Enfocament 2: Agents especialitzats (el que acabem de fer)

```python
agent_analyst = Agent(
    role="Expert Analitzador",  # Només analitza!
    tools=[read_excel_bom]
)

agent_pricer = Agent(
    role="Expert en Preus",  # Només preus!
    tools=[get_component_price]
)
```

**Avantatges:**
- ✅ Cada agent és realment expert
- ✅ Backstories específics i potents
- ✅ Cada agent fa UNA cosa molt bé
- ✅ Fàcil afegir nous agents (ex: "Expert en Alternatives")
- ✅ Fàcil canviar un agent sense afectar els altres
- ✅ Millor qualitat per tasca

### 💡 Quan usar múltiples agents?

**USA múltiples agents quan:**
- ✅ Les tasques són prou diferents (anàlisi vs preus vs informes)
- ✅ Cada tasca necessita expertesa específica
- ✅ El workflow té fases clares
- ✅ Vols escalabilitat (afegir més agents després)

**USA un sol agent quan:**
- ✅ La tasca és simple i homogènia
- ✅ No hi ha fases diferenciades
- ✅ Vols simplicitat màxima

**Regla d'or:** Si et trobes escrivint "aquest agent fa X, Y i Z", potser necessites 3 agents!

---
## Reflexió Final de la Part 2

### 🎓 Què hem après?

1. ✅ **Tools i Function Calling**
   - Per què els agents necessiten tools
   - Com crear tools personalitzades amb `@tool`
   - Com els agents decideixen quan usar-les

2. ✅ **Tools pràctiques**
   - Tool per llegir Excel (fitxers reals)
   - Tool per consultar preus (API mock)
   - Agents amb múltiples tools

3. ✅ **Traces i Debugging**
   - Anatomia completa d'una trace
   - Patrons correctes vs problemàtics
   - Com debugar quan alguna cosa falla
   - Consells pràctics de debugging

4. ✅ **Múltiples Agents**
   - Per què especialitzar agents
   - Process Sequential
   - Context entre tasks
   - Avantatges vs agent únic

---
<table style="margin: 20px 0; text-align: left; width:100%; border: 2px solid #27ae60;">
    <tr>
        <td style="width: 150px; vertical-align: middle; padding: 20px;">
            <img src="https://raw.githubusercontent.com/googlefonts/noto-emoji/main/png/128/emoji_u1f6e0.png" width="120" style="display: block;" />
        </td>
        <td style="padding: 20px;">
            <h2 style="color:#27ae60; margin: 0;">Excel·lent treball!</h2>
            <p style="color:#333; margin: 10px 0;">Has completat la Part 2 i ara domines tools, traces i agents col·laboratius.</p>
            <p style="color:#333; margin: 10px 0;">Ja tens totes les peces per construir sistemes d'agents reals i potents!</p>
            <p style="color:#ff7800; margin: 10px 0 0 0;"><strong>Proper pas:</strong> Part 3 - Cas Pràctic Complet (Sistema d'Escandalls)</p>
        </td>
    </tr>
</table>